# -1. HelloWorld

In [ ]:
print("This Jupyter notebook contains (hopefully) everything necessary (if you are on Linux) to make a simple Object Detector with TensorFlow Object Detection API")

# 0. Setup Paths

In [ ]:
CUSTOM_MODEL_NAME = "my_ssd_mbnet"
WORKSPACE_PATH = "Tensorflow/workspace"
SCRIPTS_PATH = "Tensorflow/scripts"
API_MODEL_PATH = "Tensorflow/models"
ANNOTATION_PATH = WORKSPACE_PATH + "/annotations/"
IMAGE_PATH = WORKSPACE_PATH + "/images"
MODEL_PATH = WORKSPACE_PATH + "/models"
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + "/pre-trained-models"
CONFIG_PATH = MODEL_PATH + "/" + CUSTOM_MODEL_NAME + "/pipeline.config"
CHECKPOINT_PATH = MODEL_PATH + "/" + CUSTOM_MODEL_NAME

# 1. Install necessary libraries 

In [ ]:
#We will use labelImg to label our images
!cd Tensorflow/workspace && git clone https://github.com/tzutalin/labelImg.git 

In [ ]:
#PyQt5 and lxml are a necessary library for labelImg
!pip install pyqt5
!pip install lxml

In [ ]:
!cd Tensorflow/workspace/labelImg && pyrcc5 -o resources.py resources.qrc

In [ ]:
!cd Tensorflow/workspace/labelImg && mv resources.py resources.qrc libs

In [ ]:
#In order to run label Img
!cd Tensorflow/workspace/labelImg && python3 labelImg.py


# 2. Create Label Map

In [ ]:
labels = [{'name': 'mask','id':1}, {'name':'nomask', 'id':2}]

In [ ]:
labels

In [ ]:
with open(ANNOTATION_PATH + 'label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Install Tensorflow and clone models

In [ ]:
#Donwload TensorFlow models
!pip install tensorflow

!cd Tensorflow && git clone https://github.com/tensorflow/models


In [ ]:
#Next we need to install protobuf
#Download the latest version here:
#https://github.com/protocolbuffers/protobuf/releases

!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=.


In [ ]:
#cocoapi is necessary because is a dependance of Tensorflow2.x (which are using)

!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && make

#If you have a problem compiling the MakeFile, you may need to change the Makefile from <python> to <python3>

In [ ]:
#Substitute <PATH_TO_TF> with your path
#You can check the path with "pwd" on the terminal
#!cd cocoapi/PythonAPI && cp -r pycocotools <*PATH_TO_TF>/TensorFlow/models/research/
!cd cocoapi/PythonAPI && cp -r pycocotools /home/vasco/Desktop/CODE/Mask_Detector/Tensorflow/models/research


# 4. Create TF records

In [ ]:
!python3 {SCRIPTS_PATH + "/generate_tfrecord.py"} -x {IMAGE_PATH + "/train"} -l {ANNOTATION_PATH + "/label_map.pbtxt"} -o {ANNOTATION_PATH + "/train.record"}
!python3 {SCRIPTS_PATH + "/generate_tfrecord.py"} -x {IMAGE_PATH + "/test"} -l {ANNOTATION_PATH + "/label_map.pbtxt"} -o {ANNOTATION_PATH + "/test.record"}


# 5. Copy Model Config to Training Folder

In [ ]:
!mkdir {'Tensorflow/workspace/models/'+CUSTOM_MODEL_NAME}

In [ ]:
#To perform the transfer learning, download the model you like from:
#https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
#I used "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu", but you can use anything you like
!mkdir Tensorflow/workspace/pre-trained-models
#After installing extract to the new dir

In [ ]:
#If you use a different model make sure to change the next line
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

# 6. Update Config For Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)


In [ ]:
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [ ]:
pipeline_config

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

# 6. Train the model

In [ ]:
#Take the next line and paste it into the terminal on the main folder
print("""python3 {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=5000""".format(API_MODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

# 7. Load Train Model From Checkpoint

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
# Load pipeline config and build a detection model


configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [ ]:
import cv2 as cv
import numpy as np

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [ ]:
#My webcam is at 0, but if the next line doesn't work for you maybe try another number (maybe 1)
cap = cv.VideoCapture(0)
width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

In [ ]:
while True: 
    suc, img = cap.read()
    if not(suc):
        break
    image_np = np.array(img)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1,
                min_score_thresh=.7,
                agnostic_mode=False)

    cv.imshow('object detection',  image_np_with_detections)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break
cv.destroyAllWindows()